# Calculo de LTV y Cobertura Mensual 

![Guia de Valores Fasecolda](https://autosboutique.com/wp-content/uploads/2022/01/Fasecolda400-300x300.jpg)


*"Los valores de los vehículos de la Guía de valores de Fasecolda, son el resultado de un promedio de oferta de vehículo usado en el país según la oferta disponible en las diferentes fuentes del mercado clasificados, concesionarios e Internet."*


In [353]:
##CARGUE DE LIBRERIAS UTILES Y NECESARIAS
import pyodbc # Importando la librería pyodbc para conectarse a bases de datos ODBC
import pandas as pd # Importando la librería Pandas para trabajar con DataFrames
import numpy as np   # Importando la librería Numpy para operaciones matemáticas y estadísticas
import glob # Importando la librería Glob para obtener listas de archivos basadas en patrones
import re # Importando la librería re para trabajar con expresiones regulares
from alive_progress import alive_bar # Importando la barra de progreso alive_progress para mostrar  barras de progreso animadas en la consola
import datetime # Importando la librería datetime para trabajar con fechas y horas
import os # Importando la librería os para interactuar con el sistema operativo (paths, environment variables, etc.)
import timeit
import time
from datetime import datetime, timedelta, date
import calendar
pd.set_option('display.max_columns', None)

In [354]:
inicio = time.time()
time.sleep(1)

In [355]:
# Funciones

def first_day(d):
    if d.month == 1:
        return d.replace(month =12, year = d.year -1, day= 1)
    else:
        return d.replace(month= d.month -1, year= d.year, day = 1)
    
def last_day(d):
    if d.month == 1:
        return d.replace(month =12, year = d.year -1, day= 31)
    elif d.month == 3:
        if calendar.isleap(d.year) == True:
            return d.replace(month =d.month -1, year = d.year, day= 29)
        else:
            return d.replace(month =d.month -1, year = d.year, day= 28)
    elif (d.month == 5) | (d.month == 7) |(d.month == 10) | (d.month == 12):
        return d.replace(month =d.month -1, year = d.year, day= 30)
    else:
        return d.replace(month= d.month -1, year= d.year, day = 31)


**Se declaran la fecha correpondiente a los dias primero y ultimo del mes inmediatamente anterior**

In [356]:
# hoy = date.today()
# primero = first_day(hoy).strftime('%Y%m%d')
# ultimo = last_day(hoy).strftime('%Y%m%d')
# ayer =datetime.today()+timedelta(days=-1)
primero = 20250130
ultimo = 20250131
ayer = '2025-05-02 14:42:16.563049'
ayer = datetime.strptime(ayer, '%Y-%m-%d %H:%M:%S.%f')
#año
a1 = 2025
print(primero, ultimo, ayer)

20250130 20250131 2025-05-02 14:42:16.563049


**Se importa la base de datos de Fact_Garantias del periodo deseado (recordar que es mes vencido)**

In [357]:
cnxn = pyodbc.connect('DRIVER=SQL Server;Server=192.168.50.38\\DW_FZ;Database=DW_FZ;Port=1433;Trusted_Connection=yes')

Base de garantias 

In [358]:
query = f"""select * 
from (SELECT *
	  ,rank() over (partition by [vehiculo_marca],credito order by id_fecha desc) rank_id
     FROM [DW_FZ].[dbo].Fact_Garantias where id_fecha = {ultimo}) a
  where rank_id=1;
"""

garantias_total = pd.read_sql_query(query, cnxn)
garantias_total.info()

C:\Users\cristian.parra\AppData\Local\Temp\ipykernel_19936\2569993497.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  garantias_total = pd.read_sql_query(query, cnxn)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 44 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id_fecha                   44898 non-null  int64         
 1   credito                    44898 non-null  float64       
 2   codigo_sucursal            44898 non-null  int64         
 3   numero_id_deudor           44898 non-null  float64       
 4   nombre_deudor              44898 non-null  object        
 5   direccion_domicilio        44898 non-null  object        
 6   ciudad                     44898 non-null  object        
 7   telefono_domicilio         44898 non-null  object        
 8   telefono_empresa           44898 non-null  object        
 9   telefono_celular           44898 non-null  object        
 10  valor_desembolso           44898 non-null  float64       
 11  fecha_desembolso           44898 non-null  datetime64[ns]
 12  fech

In [359]:
# query = f"""select * 
# from (SELECT *
# 	  ,rank() over (partition by [vehiculo_marca],credito order by id_fecha desc) rank_id
#      FROM [DW_FZ].[dbo].Fact_Garantias where id_fecha BETWEEN  {primero} AND {ultimo}) a
#   where rank_id=1;
# """

# garantias_total = pd.read_sql_query(query, cnxn)
# garantias_total.info()

In [360]:
garantias_total.tail()

,id_fecha,credito,codigo_sucursal,numero_id_deudor,nombre_deudor,direccion_domicilio,ciudad,telefono_domicilio,telefono_empresa,telefono_celular,valor_desembolso,fecha_desembolso,fecha_vencimiento,valor_saldo,dias_mora,valor_garantia,id_vehiculo,vehiculo_placa,vehiculo_marca,vehiculo_descripcion,vehiculo_color,vehiculo_modelo,vehiculo_codigo_fasecolda,vehiculo_cilindraje,vehiculo_numero_motor,vehiculo_numero_chasis,vehiculo_garantia_uso,codigo_garantia,concesionario,tipo_seguro_gps,gps_fecha_vencimiento,vlr_pri_seg_gps,numero_interno_poliza,numero_poliza,tipo_seguro_veh,ind_endo,poliza_fecha_inicio,poliza_fecha_vencimiento,valor_poliza,vlr_pri_seg_veh,memo,fecha_insercion,id_garantia,rank_id
44893,20250131,238668.0,114,901327959.0,GRUPO EMPRESARIAL RRL SAS,VEREDA SANTA BARBARA granja la caba¦a,ARBELAEZ CUNDIN,,000000000,0000000000,399002000.0,2024-09-19,2029-09-19,388474160.0,11,498000000.0,79111,NOK329,VOLQUETA Placa NOK329,VOLQUETA Marca CHEVROLET FVZ 700P FORWAR,BLANCO,2025,01626113,0,6HK1-291233,JALFVZ347S7000020,PUB,1,CENTRODIESEL,337,2029-09-19,68250.0,696628.0,,90,0,2024-09-19,2029-09-19,498000000.0,1166950.0,,2025-01-31 04:36:39.423,187268,1
44894,20250131,186543.0,614,901156430.0,CONSTRUMAQ DC SAS,CARRERA 24 No 17 - 51,SANTANDER BUCAR,,152601988,3152601988,250000000.0,2021-01-05,2026-01-05,82131962.0,25,239800000.0,31630,SZZ636,VOLQUETA Placa SZZ636,VOLQUETA INTERNATIONAL 7600 WORKSTAR ^35,BLANCO,2013,03626091,0,35300299,3HTWYAHT5DN197805,PUB,1,CASA TORO BARRANQUIL,0,1980-01-01,0.0,0.0,,0,0,1980-01-01,1980-01-01,0.0,0.0,,2025-01-31 04:36:39.423,143130,1
44895,20250131,156393.0,714,91001190.0,CHARRY MORENO HERNAN ELIACER,KR 40 3B 25,CESAR AGUACHICA,005662118,,3173425925,97800000.0,2018-10-16,2028-09-16,66548836.0,441,171500000.0,31688,TAW518,VOLQUETA Placa TAW518,VOLQUETA Marca FOTON AUMAN Modelo 2013 P,BLANCO,2013,18926001,0,1511J022507,LVBDLPJB2DH200003,PUB,1,FINANZAUTO,0,1980-01-01,0.0,0.0,,0,0,1980-01-01,1980-01-01,0.0,0.0,,2025-01-31 04:36:39.423,117801,1
44896,20250131,211786.0,714,80121226.0,PEDRAZA LOPEZ JOHN DANILO,CALLE 2A 17-27 TORRE B APTO 201,BOYACA SOGAMOSO,002111111,,3116216529,242100000.0,2022-06-30,2027-07-01,170664241.0,0,322100000.0,56363,TLP889,VOLQUETA Placa TLP889,VOLQUETA Marca HINO FM1A L7D ^500! MT 89,BLANCO,2018,03726015,0,A09CUP10132,9F3FM1AL7JXX10039,PUB,1,DIRECTO,337,2027-07-01,68250.0,531118.0,2000243165,292,1,2022-06-30,2023-06-30,322100000.0,0.0,,2025-01-31 04:36:39.423,164195,1
44897,20250131,217053.0,614,64587424.0,BALOCO BENITEZ YENIS YOJANA,CL 65 47 45 PI APTO 1,BARRANQUILLA AT,022218893,,3022218893,85000000.0,2022-12-12,2028-12-12,67997097.0,0,85000000.0,82111,NUL893,VOLSKWAGEN NUL893,VOLSKWAGEN TAOS COMNFORTLINE SELECT 2024,PLATA,2024,,0,DSJ109476,8AWBJ6B25RA812076,,1,,0,1980-01-01,0.0,730960.0,220037844-,30,0,2024-11-12,2028-12-12,366490000.0,1401254.0,,2025-01-31 04:36:39.423,190348,1


In [361]:
garantias_total = garantias_total[["credito", "vehiculo_codigo_fasecolda", "vehiculo_placa", "vehiculo_modelo", "valor_garantia"]]

In [362]:
garantias_total.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   credito                    44898 non-null  float64
 1   vehiculo_codigo_fasecolda  44898 non-null  object 
 2   vehiculo_placa             44898 non-null  object 
 3   vehiculo_modelo            44898 non-null  int64  
 4   valor_garantia             44898 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.7+ MB


Se importan los datos de la basa fasecolda

In [363]:
query = f"""
SELECT [Codigo], [Homologocodigo], [Marca], [Clase], [Referencia1], [Referencia2], [Referencia3], [Um],
[1970], [1971], [1972], [1973], [1974], [1975], [1976], [1977], [1978], [1979], [1980], [1981], [1982], 
[1983], [1984], [1985], [1986], [1987], [1988], [1989], [1990], [1991], [1992], [1993], [1994], [1995], 
[1996], [1997], [1998], [1999], [2000], [2001], [2002], [2003], [2004], [2005], [2006], [2007], [2008], 
[2009], [2010], [2011], [2012], [2013], [2014], [2015], [2016], [2017], [2018], [2019], [2020], [2021], 
[2022], [2023], [2024], [2025], [2026] 
FROM [Analitica].[dbo].[Guia_fasecolda_nuevo_formato];
 ;
""" 
GuiaFasecolda = pd.read_sql_query(query,cnxn)
GuiaFasecolda.info()

C:\Users\cristian.parra\AppData\Local\Temp\ipykernel_19936\1636760040.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  GuiaFasecolda = pd.read_sql_query(query,cnxn)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17374 entries, 0 to 17373
Data columns (total 65 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Codigo          17374 non-null  object
 1   Homologocodigo  17374 non-null  object
 2   Marca           17374 non-null  object
 3   Clase           17374 non-null  object
 4   Referencia1     17374 non-null  object
 5   Referencia2     17374 non-null  object
 6   Referencia3     17374 non-null  object
 7   Um              17374 non-null  object
 8   1970            17374 non-null  object
 9   1971            17374 non-null  object
 10  1972            17374 non-null  object
 11  1973            17374 non-null  object
 12  1974            17374 non-null  object
 13  1975            17374 non-null  object
 14  1976            17374 non-null  object
 15  1977            17374 non-null  object
 16  1978            17374 non-null  object
 17  1979            17374 non-null  object
 18  1980  

merge de guia_fasecolda con garatias 

In [364]:
# Realizamos el merge entre los DataFrames garantias_total y GuiaFasecolda
df_merged_garantias_guia = pd.merge(garantias_total, GuiaFasecolda, left_on="vehiculo_codigo_fasecolda", right_on="Codigo", how="left")
df_merged_garantias_guia



,credito,vehiculo_codigo_fasecolda,vehiculo_placa,vehiculo_modelo,valor_garantia,Codigo,Homologocodigo,Marca,Clase,Referencia1,Referencia2,Referencia3,Um,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026
0,38842.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,41182.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,64743.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,72451.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,73048.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44893,238668.0,01626113,NOK329,2025,498000000.0,01626113,01665044,CHEVROLET,VOLQUETA,FVZ,700P FORWARD [CORTO] Euro IV,MT 7800CC TD 6X4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,243500,253600,264200,275200,286700,298600,320400,384600,0,0,500400
44894,186543.0,03626091,SZZ636,2013,239800000.0,03626091,03665017,INTERNATIONAL,VOLQUETA,7600,WORKSTAR [350HP],MT TD 6X4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,187200,199200,211900,225400,239800,255100,271400,0,0,0,0,0,0,0,0,0,0,323100
44895,156393.0,18926001,TAW518,2013,171500000.0,18926001,18965001,FOTON,VOLQUETA,AUMAN,BJ3253DLPJB-7,MT 9700CC TD 6X4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,147100,154800,162900,171500,180500,190000,0,0,0,0,0,0,0,0,0,0,275000
44896,211786.0,03726015,TLP889,2018,322100000.0,03726015,03765014,HINO,VOLQUETA,FM1A,L7D [500],MT 8900CC TD 6X4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,322100,339100,356900,375700,403100,481400,0,0,481400


Se trae el historico de la Fact_Cartera_Mes, con el fin de obtener la garantia del primer comite 

In [365]:

query = """
WITH CTE AS (
    SELECT 
        credito, 
        valor_garantia,
        modelo_vehiculo,
        ROW_NUMBER() OVER (PARTITION BY credito ORDER BY id_fecha) AS row_num
    FROM Fact_Cartera_Mes
)
SELECT 
    credito, 
    valor_garantia,
    modelo_vehiculo
FROM CTE
WHERE row_num = 1
ORDER BY credito;
"""
hist_cartera = pd.read_sql(query, cnxn)
hist_cartera.info

C:\Users\cristian.parra\AppData\Local\Temp\ipykernel_19936\46807163.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  hist_cartera = pd.read_sql(query, cnxn)


<bound method DataFrame.info of          credito  valor_garantia  modelo_vehiculo
0         7285.0      12814000.0             1986
1        10449.0             0.0             1993
2        13548.0      11000000.0             1994
3        13745.0             0.0             1991
4        17160.0      28695000.0             1993
...          ...             ...              ...
200900  244520.0     175990000.0             2025
200901  244521.0      84100000.0             2021
200902  244522.0     104990000.0             2025
200903  398036.0      75000000.0             2005
200904  935994.0      53500000.0             2005

[200905 rows x 3 columns]>

Merge de guia_fasecolda & garantias con cartera_mes

In [366]:
df_merged = pd.merge(df_merged_garantias_guia, hist_cartera, left_on= "credito", right_on ="credito", how="left")
df_merged

,credito,vehiculo_codigo_fasecolda,vehiculo_placa,vehiculo_modelo,valor_garantia_x,Codigo,Homologocodigo,Marca,Clase,Referencia1,Referencia2,Referencia3,Um,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,valor_garantia_y,modelo_vehiculo
0,38842.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24800000.0,2006.0
1,41182.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60400000.0,2006.0
2,64743.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62500000.0,2010.0
3,72451.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65500000.0,2011.0
4,73048.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33190000.0,2011.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44893,238668.0,01626113,NOK329,2025,498000000.0,01626113,01665044,CHEVROLET,VOLQUETA,FVZ,700P FORWARD [CORTO] Euro IV,MT 7800CC TD 6X4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,243500,253600,264200,275200,286700,298600,320400,384600,0,0,500400,498000000.0,2025.0
44894,186543.0,03626091,SZZ636,2013,239800000.0,03626091,03665017,INTERNATIONAL,VOLQUETA,7600,WORKSTAR [350HP],MT TD 6X4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,187200,199200,211900,225400,239800,255100,271400,0,0,0,0,0,0,0,0,0,0,323100,334390000.0,2020.0
44895,156393.0,18926001,TAW518,2013,171500000.0,18926001,18965001,FOTON,VOLQUETA,AUMAN,BJ3253DLPJB-7,MT 9700CC TD 6X4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,147100,154800,162900,171500,180500,190000,0,0,0,0,0,0,0,0,0,0,275000,122800000.0,2013.0
44896,211786.0,03726015,TLP889,2018,322100000.0,03726015,03765014,HINO,VOLQUETA,FM1A,L7D [500],MT 8900CC TD 6X4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,322100,339100,356900,375700,403100,481400,0,0,481400,322100000.0,2018.0


depreciación 

In [367]:
# Aseguramos que las columnas necesarias sean numéricas
df_merged["valor_garantia_y"] = pd.to_numeric(df_merged["valor_garantia_y"], errors='coerce')

columns_years = [str(year) for year in range(1970, 2027)]

# Crear una nueva columna 'valor_por_ano' para almacenar el valor de la columna correspondiente al año
df_merged["valor_por_ano"] = None

# Iterar sobre las filas del DataFrame combinado
for index, row in df_merged.iterrows():
    # Obtener el año de la columna 'vehiculo_modelo'
    year = str(row['vehiculo_modelo'])  # Convertir el año a string para la comparación
    
    # Verificar si el año está en las columnas de los años
    if year in columns_years:
        # Asignar el valor de la columna correspondiente al año
        df_merged.at[index, 'valor_por_ano'] = row[year]
    

# Aseguramos que la columna 'valor_por_ano' esté como numérica
df_merged['valor_por_ano'] = pd.to_numeric(df_merged['valor_por_ano'], errors='coerce')

# Multiplicamos 'valor_por_ano' por 1000
df_merged['valor_por_ano'] = df_merged['valor_por_ano'] * 1000



In [368]:
# En caso de que vehivulo modelo sea igual a  0 o vacio se toma el valro de modelo_vehiculo que viene de fact_cartera_mes
a1 = 2025  # año actual

# Crear la nueva columna 'valor_calculado' con la fórmula y redondear a 2 decimales
df_merged["valor_calculado"] = df_merged.apply(
    lambda row: round(min(0.05 * (a1 - (row['vehiculo_modelo'] if row['vehiculo_modelo'] not in [0, None, ''] else row['modelo_vehiculo'])), 1), 2) 
    if a1 > (row['vehiculo_modelo'] if row['vehiculo_modelo'] not in [0, None, ''] else row['modelo_vehiculo']) 
    else 0,
    axis=1
)

In [369]:
#prueba 
df_merged["valor_depreciado"] = df_merged.apply(
    lambda row: round(
        row['valor_garantia_y'] * (1 - row['valor_calculado']), 2
    ) if pd.isna(row['valor_por_ano'])  # Si valor_por_ano está vacío
    else round(
        row['valor_garantia_y'] * 0.75, 2  # Si valor_por_ano es 0, deprecia 25% de valor_garantia_y
    ) if row['valor_por_ano'] == 0  
    else round(
        row['valor_por_ano'] * 0.75, 2  # Si valor_por_ano tiene valor distinto de 0, deprecia 25% de valor_por_ano
    ), 
    axis=1
)



df_merged

,credito,vehiculo_codigo_fasecolda,vehiculo_placa,vehiculo_modelo,valor_garantia_x,Codigo,Homologocodigo,Marca,Clase,Referencia1,Referencia2,Referencia3,Um,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,valor_garantia_y,modelo_vehiculo,valor_por_ano,valor_calculado,valor_depreciado
0,38842.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24800000.0,2006.0,NaN,0.95,1240000.0
1,41182.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60400000.0,2006.0,NaN,0.95,3020000.0
2,64743.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62500000.0,2010.0,NaN,0.75,15625000.0
3,72451.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65500000.0,2011.0,NaN,0.70,19650000.0
4,73048.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33190000.0,2011.0,NaN,0.70,9957000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44893,238668.0,01626113,NOK329,2025,498000000.0,01626113,01665044,CHEVROLET,VOLQUETA,FVZ,700P FORWARD [CORTO] Euro IV,MT 7800CC TD 6X4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,243500,253600,264200,275200,286700,298600,320400,384600,0,0,500400,498000000.0,2025.0,0.0,0.00,373500000.0
44894,186543.0,03626091,SZZ636,2013,239800000.0,03626091,03665017,INTERNATIONAL,VOLQUETA,7600,WORKSTAR [350HP],MT TD 6X4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,187200,199200,211900,225400,239800,255100,271400,0,0,0,0,0,0,0,0,0,0,323100,334390000.0,2020.0,239800000.0,0.60,179850000.0
44895,156393.0,18926001,TAW518,2013,171500000.0,18926001,18965001,FOTON,VOLQUETA,AUMAN,BJ3253DLPJB-7,MT 9700CC TD 6X4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,147100,154800,162900,171500,180500,190000,0,0,0,0,0,0,0,0,0,0,275000,122800000.0,2013.0,171500000.0,0.60,128625000.0
44896,211786.0,03726015,TLP889,2018,322100000.0,03726015,03765014,HINO,VOLQUETA,FM1A,L7D [500],MT 8900CC TD 6X4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,322100,339100,356900,375700,403100,481400,0,0,481400,322100000.0,2018.0,322100000.0,0.35,241575000.0


In [370]:

# Imprimir el DataFrame con la nueva columna 'valor_calculado'
print(df_merged[["vehiculo_codigo_fasecolda", "vehiculo_modelo", "valor_calculado"]].head())


  vehiculo_codigo_fasecolda  vehiculo_modelo  valor_calculado
0                                          0             0.95
1                                          0             0.95
2                                          0             0.75
3                                          0             0.70
4                                          0             0.70


**Ahora se importan los datos de Cartera Mes**

In [371]:
df_merged

,credito,vehiculo_codigo_fasecolda,vehiculo_placa,vehiculo_modelo,valor_garantia_x,Codigo,Homologocodigo,Marca,Clase,Referencia1,Referencia2,Referencia3,Um,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,valor_garantia_y,modelo_vehiculo,valor_por_ano,valor_calculado,valor_depreciado
0,38842.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24800000.0,2006.0,NaN,0.95,1240000.0
1,41182.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60400000.0,2006.0,NaN,0.95,3020000.0
2,64743.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62500000.0,2010.0,NaN,0.75,15625000.0
3,72451.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65500000.0,2011.0,NaN,0.70,19650000.0
4,73048.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33190000.0,2011.0,NaN,0.70,9957000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44893,238668.0,01626113,NOK329,2025,498000000.0,01626113,01665044,CHEVROLET,VOLQUETA,FVZ,700P FORWARD [CORTO] Euro IV,MT 7800CC TD 6X4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,243500,253600,264200,275200,286700,298600,320400,384600,0,0,500400,498000000.0,2025.0,0.0,0.00,373500000.0
44894,186543.0,03626091,SZZ636,2013,239800000.0,03626091,03665017,INTERNATIONAL,VOLQUETA,7600,WORKSTAR [350HP],MT TD 6X4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,187200,199200,211900,225400,239800,255100,271400,0,0,0,0,0,0,0,0,0,0,323100,334390000.0,2020.0,239800000.0,0.60,179850000.0
44895,156393.0,18926001,TAW518,2013,171500000.0,18926001,18965001,FOTON,VOLQUETA,AUMAN,BJ3253DLPJB-7,MT 9700CC TD 6X4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,147100,154800,162900,171500,180500,190000,0,0,0,0,0,0,0,0,0,0,275000,122800000.0,2013.0,171500000.0,0.60,128625000.0
44896,211786.0,03726015,TLP889,2018,322100000.0,03726015,03765014,HINO,VOLQUETA,FM1A,L7D [500],MT 8900CC TD 6X4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,322100,339100,356900,375700,403100,481400,0,0,481400,322100000.0,2018.0,322100000.0,0.35,241575000.0


#### cuando el codigo inicie en 888 y 999 no tienen garantias y lo que queda restante no se puede identificar.

## Carrocerias 

In [372]:
cyc = f"""Select t3.credito,
t3.id_fecha_desembolso,
LEFT(t3.id_fecha_desembolso, 6) AS anio
,t3.placa_vehiculo,
t4.precio_carroceria,
t4.precio_chasis,
t4.clasevehiculo,
t4.carroceria
from
(select t1.*,t2.numero_solicitud from
(select credito, placa_vehiculo, id_fecha_desembolso from dw_fz.dbo.Fact_Cartera_mes where id_fecha = 20250131 ) t1
left join (select credito, numero_solicitud from [DW_FZ].[dbo].[Fact_Cartera_Solicitud] group by credito,numero_solicitud)as t2 on t1.credito = t2.credito) t3
left join (select id_solicitud, precio_carroceria, precio_chasis, clasevehiculo, placa, carroceria
from [DW_FZ].[sol].[Dim_Sol_Vehiculos]) as t4 on t3.numero_solicitud = t4.id_solicitud and t3.placa_vehiculo = t4.placa
where t4.precio_carroceria >0 and ((t4.clasevehiculo != 'CAMIONETA PASAJ.' and (carroceria != 'VAN' or carroceria != 'FURGON')) or t4.clasevehiculo != 'AUTOMOVIL') order by t3.credito
"""
car_chas= pd.read_sql_query(cyc,cnxn)
car_chas.info()

C:\Users\cristian.parra\AppData\Local\Temp\ipykernel_19936\2173901832.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  car_chas= pd.read_sql_query(cyc,cnxn)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7840 entries, 0 to 7839
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   credito              7840 non-null   float64
 1   id_fecha_desembolso  7840 non-null   object 
 2   anio                 7840 non-null   object 
 3   placa_vehiculo       7840 non-null   object 
 4   precio_carroceria    7840 non-null   int64  
 5   precio_chasis        7840 non-null   int64  
 6   clasevehiculo        7840 non-null   object 
 7   carroceria           7840 non-null   object 
dtypes: float64(1), int64(2), object(5)
memory usage: 490.1+ KB


In [ ]:
# Convertimos la columna 'id_fecha_desembolso' en formato 'YYYYMMDD' a formato datetime
car_chas['id_fecha_desembolso'] = pd.to_datetime(car_chas['id_fecha_desembolso'], format='%Y%m%d')

# Fecha de hoy
hoy = datetime.today()

# Calculamos la diferencia en años
car_chas['anios_cumplidos'] = (hoy - car_chas['id_fecha_desembolso']).dt.days // 365


,credito,id_fecha_desembolso,anio,placa_vehiculo,precio_carroceria,precio_chasis,clasevehiculo,carroceria,anios_cumplidos
0,168303.0,2019-06-28,201906,GFP841,33341000,140000000,CAMION,FURGON,5
1,168373.0,2019-06-28,201906,GET582,14200000,55990000,CAMION,ESTACAS,5
2,168425.0,2019-06-28,201906,GET645,35000000,46990000,CAMION,furgon,5
3,168539.0,2019-06-28,201906,GET545,23000000,51990000,CAMION,ESTACAS,5
4,169205.0,2019-07-22,201907,GET713,12500000,53899999,CAMION,ESTACAS,5


In [382]:
#depreciación de las carrocerías 

# Calcular la columna 'tiempo_depre' con la lógica del vehículo
car_chas['tiempo_depre'] = np.where(car_chas['clasevehiculo'] == 'BUS / BUSETA / MICROBUS', 20, 
                                     np.where(car_chas['carroceria'] == 'TERMO', 7, 10))

# Calcular 'vl_cria_depre' de manera eficiente
car_chas['vl_cria_depre'] = car_chas['precio_carroceria'] / car_chas['tiempo_depre']

# Calcular 'depreciado_hoy' como la multiplicación de 'anios_cumplidos' y 'vl_cria_depre'
car_chas['depreciado_hoy'] = car_chas['anios_cumplidos'] * car_chas['vl_cria_depre']

# Usar np.where para asignar el valor final de la carrocería
car_chas['valor_final_carroceria'] = np.where(car_chas['depreciado_hoy'] == 0, 
                                              car_chas['precio_carroceria'], 
                                              car_chas['precio_carroceria'] - car_chas['depreciado_hoy'])

# Convertir 'valor_final_carroceria' a entero si es necesario
car_chas['valor_final_carroceria'] = car_chas['valor_final_carroceria'].astype(int)
car_chas = car_chas[car_chas['precio_carroceria'] != 1]

# Ver las primeras filas del DataFrame para comprobar
car_chas.head()


,credito,id_fecha_desembolso,anio,placa_vehiculo,precio_carroceria,precio_chasis,clasevehiculo,carroceria,anios_cumplidos,tiempo_depre,vl_cria_depre,depreciado_hoy,valor_final_carroceria
0,168303.0,2019-06-28,201906,GFP841,33341000,140000000,CAMION,FURGON,5,10,3334100.0,16670500.0,16670500
1,168373.0,2019-06-28,201906,GET582,14200000,55990000,CAMION,ESTACAS,5,10,1420000.0,7100000.0,7100000
2,168425.0,2019-06-28,201906,GET645,35000000,46990000,CAMION,furgon,5,10,3500000.0,17500000.0,17500000
3,168539.0,2019-06-28,201906,GET545,23000000,51990000,CAMION,ESTACAS,5,10,2300000.0,11500000.0,11500000
4,169205.0,2019-07-22,201907,GET713,12500000,53899999,CAMION,ESTACAS,5,10,1250000.0,6250000.0,6250000


In [ ]:
query = f"""
SELECT id_fecha, credito, id_fecha_desembolso, tipo_id_deudor, numero_id_deudor, id_vehiculo, placa_vehiculo, 
    marca_vehiculo, linea_vehiculo, modelo_vehiculo, plazo_total, valor_desembolso, numero_dias_mora, codigo_oficial, 
    estado_cartera, valor_garantia, saldo_capital, total_deuda, tipo_producto 
FROM DW_FZ.dbo.Fact_Cartera_Mes
WHERE (id_fecha = N'{ultimo}') ;
""" 
cartera_mes = pd.read_sql_query(query,cnxn)
cartera_mes.info()

**Ahora se importan los datos de Cartera Mes**

Identificacon tipo de credito y garantia 

In [383]:
# Creacion de base con los creditos que tienen garantia hipotecaria

#Lectura de la base que contiene los creditos con garantia hipotecaria
hipotecario = pd.read_excel(r'\\192.168.40.9\garaje$\BackUps_Analitica\BACKUP IVAN\ORGANIGRAMA\LTV\LTV MENSUAL\Hipotecas\HIPOTECAS.xlsx',skiprows=1)
hipotecario = hipotecario[['CREDITO','VALOR AVALÚO','FECHA AVALÚO']] # se toman solo las columnas que resultan utiles para el proceso
hipotecario.info()

hipotecario[['CREDITO','VALOR AVALÚO']].value_counts() #Se verifica los valores que contiene respecto a las varianñes credito y el valor del avaluo

hipotecario['año_avaluo'] = pd.to_datetime(hipotecario['FECHA AVALÚO'],format='mixed')
hipotecario['año_avaluo'] = [hipotecario['año_avaluo'][i].year for i in hipotecario.index]
hipotecario['año_avaluo'] = hipotecario['año_avaluo'].replace(1970,0)
print(hipotecario.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   CREDITO       18 non-null     int64         
 1   VALOR AVALÚO  18 non-null     int64         
 2   FECHA AVALÚO  18 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(2)
memory usage: 564.0 bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   CREDITO       18 non-null     int64         
 1   VALOR AVALÚO  18 non-null     int64         
 2   FECHA AVALÚO  18 non-null     datetime64[ns]
 3   año_avaluo    18 non-null     int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 708.0 bytes
None


In [411]:
año_actual = ayer.year


vlr_apreciacion = np.where(hipotecario['año_avaluo']!=0,(((año_actual - hipotecario['año_avaluo'])*0.04)+1),0)
apreciacion = hipotecario['VALOR AVALÚO'] * vlr_apreciacion

#A la data Hipotecario se adiciona la columna con el valor de apreciacion de cada inmueble
hipotecario['vlr_avaluo_apreciado'] = apreciacion.round(2)
print(hipotecario.info())
hipotecario

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   CREDITO               18 non-null     int64         
 1   VALOR AVALÚO          18 non-null     int64         
 2   FECHA AVALÚO          18 non-null     datetime64[ns]
 3   año_avaluo            18 non-null     int64         
 4   vlr_avaluo_apreciado  18 non-null     float64       
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 852.0 bytes
None


,CREDITO,VALOR AVALÚO,FECHA AVALÚO,año_avaluo,vlr_avaluo_apreciado
0,214219,76000000,2016-11-09,2016,1.033600e+08
1,156806,3561000000,2018-10-03,2018,4.558080e+09
2,178584,112672000,2018-12-17,2018,1.442202e+08
3,169184,112800000,2019-05-31,2019,1.398720e+08
4,180396,187845000,2019-11-02,2019,2.329278e+08
5,183214,110382000,2020-08-06,2020,1.324584e+08
6,187180,1389765000,2020-11-12,2020,1.667718e+09
7,188717,148816821,2020-12-24,2020,1.785802e+08
8,217808,883284000,2019-01-08,2019,1.095272e+09
9,194613,1037680000,2019-01-08,2019,1.286723e+09


## Correción 
de fact cartera_mes tomar valor_garantia a tomar del primer comite de cada credito  y depreciarlo un 
25 % y de lo que no cruzaron con fasecolda una depreciasion del 5 % anual 

sacar marca, descripción, modelo_vehiculo de la fact_cartera_mes 